In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
import numpy as np
import gym
from vqc.vqc_circuits import SkolikSchuld
import cirq
import tensorflow_quantum as tfq
from data_analysis.analysis_functions import Analysis

2023-10-16 15:48:01.010682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-16 15:48:01.010720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-16 15:48:02.968408: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-16 15:48:02.968463: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Rodrigo-PC): /proc/driver/nvidia/version does not exist
2023-10-16 15:48:02.968750: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX

In [2]:
class DataReupPQC(tf.keras.layers.Layer):

    def __init__(self, vqc, observables, activation, weights, name="MyPQC"):
        super(DataReupPQC, self).__init__(name=name)

        circuit, theta_symbols, input_symbols = vqc.circuit, vqc.parameters, vqc.inputs
        self.n_layers = vqc.num_layers

        self.rotation_weights = weights[0]
        self.input_weights = weights[1]
        self.output_weights = weights[2]

        self.theta = tf.Variable(initial_value=self.rotation_weights, trainable=False, name="thetas")
        
        self.lmbd = tf.Variable(initial_value=self.input_weights, trainable=False, name="lambdas")
        
        # Define explicit symbol order,
        symbols = [str(symb) for symb in theta_symbols + input_symbols]
        self.indices = tf.constant([symbols.index(a) for a in sorted(symbols)])
        
        self.activation = activation
        self.empty_circuit = tfq.convert_to_tensor([cirq.Circuit()])
        self.computation_layer = tfq.layers.ControlledPQC(circuit, observables)

    def call(self, inputs):
        #Inputs is a list of tensors, the first one is the input data with shape (batch_size, state_dim)
        # Batch dim gives the dimension of the batch (16,32,etc)
        batch_dim = tf.gather(tf.shape(inputs[0]), 0)

        #tiled_up_circuits tiles the required number of circuits for the batch size
        tiled_up_circuits = tf.repeat(self.empty_circuit, repeats=batch_dim)

        #tiled_up_thetas tiles the required number of thetas for the batch size
        tiled_up_thetas = tf.tile(self.theta, multiples=[batch_dim, 1])

        #tiled_up_inputs tiles the required number of inputs (states) for the number of layers in the case of data reup
        tiled_up_inputs = tf.tile(inputs[0], multiples=[1, self.n_layers])

        scaled_inputs = tf.einsum("i,ji->ji", self.lmbd, tiled_up_inputs)

        squashed_inputs = tf.atan(scaled_inputs)

        joined_vars = tf.concat([tiled_up_thetas, squashed_inputs], axis=1)
        joined_vars = tf.gather(joined_vars, self.indices, axis=1)

        output = self.computation_layer([tiled_up_circuits, joined_vars])
        
        return output
    

class LocalSkolikRescaling(tf.keras.layers.Layer):
    def __init__(self, weights):
        super(LocalSkolikRescaling, self).__init__()
        self.w = tf.Variable(
            initial_value=weights[2], dtype="float32",
            trainable=False, name="obs-weights")

    def call(self, inputs):
        return tf.math.multiply((inputs+1)/2, tf.repeat(self.w,repeats=tf.shape(inputs)[0],axis=0))
    
def QuantumQLearningAgent(vqc, quantum_model, observables, target,state_dim, rescaling_type, activation,weights):
    input_tensor = tf.keras.Input(shape=(state_dim, ), dtype=tf.dtypes.float32, name='input')
    pqc = quantum_model(vqc, observables, activation, weights)([input_tensor])
    process = tf.keras.Sequential([rescaling_type(weights)], name=target*"Target"+"Q-values")
    Q_values = process(pqc)
    model = tf.keras.Model(inputs=[input_tensor], outputs=Q_values)
    return model

In [3]:
# Parameters for the VQC
model_quantum = True
num_qubits = 4
num_layers = 5
vqc = SkolikSchuld(num_qubits, num_layers)
qubits = cirq.GridQubit.rect(1, num_qubits)
ops = [cirq.Z(q) for q in qubits]
observables = [ops[0]*ops[1], ops[2]*ops[3]]
rescaling_type = LocalSkolikRescaling
state_dim = 4
activation = "linear"

path = "../results/thesis/1.1/skolik_datareup"
skolik_datareup = Analysis(path)
weights = skolik_datareup.get_final_weights()[2]

quantum_model = DataReupPQC
model = QuantumQLearningAgent(vqc, quantum_model, observables, False, state_dim, rescaling_type, activation, weights)

def select_action(state, model):
    state_array = np.array(state) 
    state = tf.convert_to_tensor([state_array])
    q_vals = model([state])
    action = int(tf.argmax(q_vals[0]).numpy())
    return action

In [8]:
from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

In [7]:
env = gym.make('CartPole-v0')
frames = []
    
state = env.reset()
done = False
while not done:
    env.render()
    action = select_action(state, model)
    state, reward, done, info = env.step(action)
env.close()

np.save("cartpole_frames.npy", frames)

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [2]:
import cv2
import numpy as np

# Load frames from npy file
frames = np.load('acrobot_frames.npy')

# Get height and width from the first frame
height, width, _ = frames[0].shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can try other codecs as well
out = cv2.VideoWriter('acrobot.mp4', fourcc, 30.0, (width, height))

# Write frames to the video file
for frame in frames:
    out.write(frame)

# Release the VideoWriter and close any open windows
out.release()
cv2.destroyAllWindows()

